In [ ]:
!pip install langchain_openai langgraph langchain_community PyGithub flask

In [1]:
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Instantiate GitHub Toolkit
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())
tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
act_llm = ChatOpenAI(model="o1-mini", temperature=0)

agent_executor = create_react_agent(llm, tools)

act_agent = create_react_agent(act_llm, tools)

def ai(query: str):
    # System prompt allows for broad range of instructions. But what does user input look like?
    events = agent_executor.stream(
        {"messages": [("system",
                       "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                       "All output must be in HTML format and will be displayed to an end user. "
                       "You need to start with doctype and html tags and provide the entire page"),
                      ("user", query)]},
        stream_mode="values",
    )
    data = ""
    for event in events:
        event["messages"][-1].pretty_print()
        data = event["messages"][-1]

    return data.content

def ai_act(query: str):
    events = agent_executor.stream(
        {"messages": [("system",
                       "You are a web server. Do not use markdown for formatting. Do exactly what the user asks. Do not use markdown in your response. Produce minimal output."),
                      ("user", query)]},
        stream_mode="values",
    )
    data = ""
    for event in events:
        event["messages"][-1].pretty_print()
        data = event["messages"][-1]

    return data.content


In [4]:
from pathlib import Path

from dotenv import load_dotenv
from flask import Flask, request

# Load environment variables from .env
load_dotenv()

app = Flask(__name__)

@app.route('/', methods=['GET'])
def get_data():
    # Wow where is the logic?
    return ai(Path("my_app-step-4.txt").read_text())

@app.route('/act', methods=['POST'])
def act():
    payload = request.data.decode('utf-8')
    print(request)
    return ai_act(payload)


## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~

In [ ]:
app.run(debug=False, port=5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


================================ Human Message =================================

Show me all issues that I can comment on.
For each issue Show the following:
1. List of all comments on that issue
2. A text box and a button.
The button will submit this comment to an issue.

When you press the button, it should make a request to http://127.0.0.1:5001/act
The payload should include the text of the textbox, the GitHub issue id, and be presented as a prompt to another agent.
It needs to explain to the Agent that we are commenting on the issue.

At the bottom of the page have a status bar that shows response from the "/act" endpoint

The "/act" endpoint takes in a plain text prompt that looks like this:
Write a comment for the issue.
Issue ID: 2, Comment: hello

The response from the "/act" endpoint is HTML, not JSON
================================== Ai Message ==================================
Tool Calls:
  get_issues (call_kaou7dZVBwoC3Pv6Jx4jtYJE)
 Call ID: call_kaou7dZVBwoC3Pv6Jx4jtYJ

127.0.0.1 - - [26/Mar/2025 11:19:44] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues</title>
    <style>
        .issue {
            border: 1px solid #ccc;
            padding: 10px;
            margin-bottom: 20px;
        }
        .comments {
            margin-top: 10px;
            padding-left: 20px;
        }
        .comment-box {
            margin-top: 10px;
        }
        .status-bar {
            margin-top: 20px;
            padding: 10px;
            background-color: #f0f0f0;
            border: 1px solid #ccc;
        }
    </style>
    <script>
        function submitComment(issueId) {
            const commentText = document.getElementById('comment-text-' + issueId).value;
            const payload = `Write a comment for the issue.\nIssue ID: ${issueId}, Comment: ${commentText}`;
            
         

127.0.0.1 - - [26/Mar/2025 11:20:04] "POST /act HTTP/1.1" 200 -


================================== Ai Message ==================================

The comment "5th comment here" has been added to issue ID 1.
